# Using NBA Shooting Data: Considers shot type, shot clock, dribbles, closest defender distance and touch time
# Find Shot Quality for each criterion separately and then average

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import playerdashptshots, leaguedashplayerbiostats

pd.options.mode.chained_assignment = None

data_DIR = "../fdata/"

In [ ]:
league = "NBA"
player_dict = get_players_pbp(league=league)
team_data = pd.read_csv("../data/NBA_teams_database.csv")
teams_list = team_data["TeamID"].tolist()
team_dict1 = team_data.to_dict(orient="records")
teams_dict = {team["TeamID"]: team["Team"] for team in team_dict1}

In [ ]:
season = "2023"
dft = pd.read_parquet(data_DIR + f"NBA_Player_BoxScores_Base_"+season+".parquet", columns = ["PLAYER_ID","TEAM_ID"])

In [ ]:
dash_types = ["overall","shot_type","shot_clock","dribble","closest_def","closest_def_10","touch_time","closest_def_10"]
col_names = ["SHOT_TYPE","SHOT_TYPE","SHOT_CLOCK_RANGE","DRIBBLE_RANGE","CLOSE_DEF_DIST_RANGE","CLOSE_DEF_DIST_RANGE","TOUCH_TIME_RANGE","CLOSE_DEF_DIST_RANGE"]

In [ ]:
dfb = []
for i in range(0,7):
    df = pd.read_parquet(data_DIR + f"{league}_Shots_{season}_{dash_types[i]}.parquet")
    df = df.drop(columns=["FG_PCT","EFG_PCT","FG2_PCT","FG3_PCT","GP","G"])
    dfb.append(df)
dfm = pd.merge(dfb[4],dfb[5], on = ["PLAYER_ID","PLAYER_NAME_LAST_FIRST","CLOSE_DEF_DIST_RANGE"], suffixes=["","_x"])
dfm["FGM"] =  dfm["FGM"] -  dfm["FGM_x"]
dfm["FG2M"] = dfm["FG2M"] - dfm["FG2M_x"]
dfm["FG3M"] = dfm["FG3M"] - dfm["FG3M_x"]
dfm["FGA"] =  dfm["FGA"] -  dfm["FGA_x"]
dfm["FG2A"] = dfm["FG2A"] - dfm["FG2A_x"]
dfm["FG3A"] = dfm["FG3A"] - dfm["FG3A_x"]
dfm = dfm.drop(dfm.filter(regex = '_x').columns, axis=1)
dfb.append(dfm)

In [ ]:
dfa = []
for i in range(0,8):
    if i in [0,4]:
        continue
    df = dfb[i]
    df_avg = df.groupby(col_names[i])[["FG2M","FG2A","FG3M","FG3A"]]\
        .agg({"FG2M": ["sum"],"FG2A": ["sum"],"FG3M": ["sum"],"FG3A": ["sum"]})
    df_avg.columns = ["FG2M","FG2A","FG3M","FG3A"]
    df_avg["xFG2"] = (df_avg["FG2M"] / df_avg["FG2A"]).round(3)
    df_avg["xFG3"] = (df_avg["FG3M"] / df_avg["FG3A"]).round(3)

    df_tm = df_avg.reset_index()
    df_tm = df_tm.drop(columns= ["FG2M","FG2A","FG3M","FG3A"])
    shots = pd.merge(df,df_tm,on=col_names[i])
    shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
    shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
    shots = shots.replace([np.inf, -np.inf], np.nan)
    shots = shots.fillna(0)

    shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
    shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)
    fg1 = (shots
        .groupby(['PLAYER_ID'])[['FGM', 'FGA', 'PTS', 'xPTS']]
        .agg({'FGM': ["sum"], 'FGA': ["sum"], 'PTS': ["sum"], 'xPTS': ["sum"]}))
    fg1.columns = ['FGM', 'FGA', 'PTS', 'xPTS']
    fg1['eFG'] = np.round(fg1['PTS']/fg1['FGA']/2, 3)
    fg1['xeFG'] = np.round(fg1['xPTS']/fg1['FGA']/2, 3)
    fg1['Shot_Making'] = np.round((fg1['PTS'] - fg1['xPTS'])/fg1['FGA'], 3)
    fg1 = fg1.fillna(0)
    fg1["Points_Added"] = fg1["Shot_Making"]* fg1["FGA"]
    fg1 = fg1.reset_index()
    fg1["Player"] = fg1["PLAYER_ID"].map(player_dict)
    fg1.insert(1,"Player",fg1.pop("Player"))
    # fg1 = fg1.drop(columns= ["PLAYER_ID"])
    dfa.append(fg1)

In [ ]:
fg = pd.concat(dfa)
fg = fg.groupby("PLAYER_ID")[fg.columns[2:]]\
        .agg({"FGM":["max"],"FGA":["max"],
              "PTS":["max"], "xPTS":["max"],
              "eFG":["mean"],"xeFG":["mean"],"Shot_Making":["mean"],"Points_Added":["mean"],
              })
fg.columns = ["FGM","FGA","PTS", "xPTS","eFG","xeFG","Shot_Making","Points_Added"]
fg = fg.reset_index()
fg["Player"] = fg["PLAYER_ID"].map(player_dict)
fg.insert(1,"Player",fg.pop("Player"))
fg["Points_Added"] = fg["Shot_Making"]* fg["FGA"]
fg[['FGM', 'FGA', 'PTS']] = fg[['FGM', 'FGA', 'PTS']].round(0).astype(int)
fg[['xPTS', 'Points_Added']] = fg[['xPTS', 'Points_Added']].round(1)
fg[['eFG', 'xeFG']] = fg[['eFG', 'xeFG']].round(3)
fg[['Shot_Making']] = fg[['Shot_Making']].round(3)
fg = pd.merge(fg,dft,on="PLAYER_ID")
fg["Team"] = fg["TEAM_ID"].map(teams_dict)
fg.insert(2,"Team",fg.pop("Team"))
fg = fg.drop(columns=["TEAM_ID"])

In [ ]:
fg.query("Player == 'Davion Mitchell'")

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 100").sort_values(by="Points_Added",ascending=False).reset_index(drop=True)

In [ ]:
export_DIR = "../../repos/csv/"

In [ ]:
fg

In [ ]:
# df_e.to_csv(export_DIR + "NBA_Shot_Quality.csv")

In [ ]:
df_e.to_csv("NBA_Shot_Quality_V1.csv")

In [ ]:
df_e.query("Player == 'Davion Mitchell'")

In [ ]:
fg.query("PTS > 100").sort_values(by="PTS",ascending=False).head(20)

In [ ]:
fg.query("PTS > 100").sort_values(by="Shot_Making",ascending=False).head(20)